In [3]:
import pandas as pd
import ast
import os

# Define the path to the CSV file
CSV_FILE_PATH = "D:\DE Projects General\synthetic-retail-data-\data\Retail_Transactions_Dataset.csv"

# --- 1. Read the source data ---
try:
    # Reading the CSV file
    df_source = pd.read_csv(CSV_FILE_PATH)
    print(f"✓ Successfully read {len(df_source)} rows from the source file.")
except FileNotFoundError:
    print(f"✗ Error: File not found at {CSV_FILE_PATH}. Exiting.")
    exit()

# --- 2. Define the cleaning function ---
def safe_literal_eval(val):
    """
    Converts a string representation of a Python list (e.g., "['Milk', 'Eggs']")
    into an actual Python list object. Handles errors by returning an empty list.
    """
    if isinstance(val, str):
        try:
            # ast.literal_eval safely evaluates the string literal
            return ast.literal_eval(val)
        except (ValueError, SyntaxError):
            # Return an empty list if the string format is invalid
            return []
    return val

# --- 3. Clean and transform the 'Product' column ---
# Apply the safe evaluation function to create a new column of actual list objects
df_source['Product_List'] = df_source['Product'].apply(safe_literal_eval)

# --- 4. Explode the DataFrame ---
# Apply the explode function on the list column to create one row per list item
# We reset the index to clean up after the explosion
df_line_items = df_source.explode('Product_List').reset_index(drop=True)

# --- 5. Finalize and clean columns ---

# Rename the exploded column to 'ProductName' for clarity
df_line_items.rename(columns={'Product_List': 'ProductName'}, inplace=True)

# Drop the original string 'Product' column as it is no longer needed
df_line_items.drop(columns=['Product'], inplace=True)

# Optional: Clean up whitespace from the new product names
df_line_items['ProductName'] = df_line_items['ProductName'].str.strip()

print("✓ Product column successfully transformed and exploded.")

# --- 6. Save the resulting DataFrame back to the CSV file ---

# The resulting file will contain the new 'ProductName' column, 
# and the original 'Product' column (which contained the list) will be removed.

try:
    df_line_items.to_csv(CSV_FILE_PATH, index=False, encoding='utf-8')
    
    # Calculate the change in row count
    original_rows = len(df_source)
    final_rows = len(df_line_items)
    
    print("\n--- Saving Results ---")
    print(f"✓ Successfully saved the exploded data back to: {CSV_FILE_PATH}")
    print(f"  - Original number of transactions (rows): {original_rows}")
    print(f"  - Final number of line items (rows) after explode: {final_rows}")
    print("  - The data is now in the 'Long Format' ready for ETL.")
    
except Exception as e:
    print(f"✗ An error occurred while saving the file: {e}")

✓ Successfully read 1000000 rows from the source file.
✓ Product column successfully transformed and exploded.

--- Saving Results ---
✓ Successfully saved the exploded data back to: D:\DE Projects General\synthetic-retail-data-\data\Retail_Transactions_Dataset.csv
  - Original number of transactions (rows): 1000000
  - Final number of line items (rows) after explode: 3000343
  - The data is now in the 'Long Format' ready for ETL.


In [5]:
df_source.dtypes

Transaction_ID         int64
Date                  object
Customer_Name         object
Product               object
Total_Items            int64
Total_Cost           float64
Payment_Method        object
City                  object
Store_Type            object
Discount_Applied        bool
Customer_Category     object
Season                object
Promotion             object
Product_List          object
dtype: object